In [2]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI

# Cargar variables de entorno
load_dotenv()

# Cliente OpenAI (usa GitHub Models o API oficial según tu .env)
client = OpenAI(
    base_url=os.getenv("OPENAI_BASE_URL"),   # Si usas GitHub Models
    api_key=os.getenv("GITHUB_TOKEN")        # Cambia por OPENAI_API_KEY si usas OpenAI directo
)

# Cargar datos EcoMarket
with open("data/ecomarket.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Historial de conversación
mensajes = [
    {
        "role": "system",
        "content": (
            "Eres el asistente virtual de EcoMarket. "
            "Debes responder sobre productos, sucursales y empleados usando el archivo JSON. "
            "Si el usuario pide algo relacionado al menú (productos, sucursales, empleados, menú), "
            "ejecuta la acción correspondiente. "
            "Si el usuario escribe algo que no corresponde, respóndele con IA pero aclara que la información no está en EcoMarket. "
            "Si no entiendes, recuérdale las opciones válidas: productos, sucursales, empleados, menú."
        )
    }
]


In [3]:
def listar_productos():
    salida = "\n📦 Productos disponibles:\n"
    for suc in data["sucursales"]:
        salida += f"\nSucursal: {suc['nombre']} ({suc['ubicacion']})\n"
        for prod in suc["productos"]:
            salida += f" - {prod['nombre']} (SKU: {prod['sku']}, Cantidad: {prod['cantidad']})\n"
    return salida

def listar_sucursales():
    salida = "\n🏬 Sucursales:\n"
    for suc in data["sucursales"]:
        salida += f"- {suc['nombre']} | Ubicación: {suc['ubicacion']}\n"
    return salida

def listar_empleados():
    salida = "\n👥 Empleados:\n"
    for suc in data["sucursales"]:
        salida += f"\nSucursal: {suc['nombre']}\n"
        for emp in suc["empleados"]:
            salida += f" - {emp}\n"
    return salida

def mostrar_menu():
    return (
        "\n=== EcoMarket Asistente Virtual ===\n"
        "1. Ver productos\n"
        "2. Ver sucursales\n"
        "3. Ver empleados\n"
        "4. Consultar al asistente (IA)\n"
        "5. Salir\n"
    )


In [5]:
def procesar_entrada(user_input):
    texto = user_input.lower()

    if "producto" in texto:
        return listar_productos()
    elif "sucursal" in texto or "tienda" in texto:
        return listar_sucursales()
    elif "empleado" in texto or "trabajador" in texto:
        return listar_empleados()
    elif "menú" in texto or "menu" in texto or "opcion" in texto:
        return mostrar_menu()
    elif texto in ["salir", "exit", "quit"]:
        return "SALIR"
    else:
        # Simula que piensa antes de responder
        print("\n🤖 Pensando...")

        mensajes.append({"role": "user", "content": user_input})
        try:
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=mensajes,
                temperature=0.3,
                max_tokens=200
            )
            reply = response.choices[0].message.content
            mensajes.append({"role": "assistant", "content": reply})
            return f"🤖 Asistente: {reply}"
        except Exception as e:
            return f"Error en la llamada a IA: {e}"


In [6]:
def main():
    print("👋 ¡Bienvenido! Soy el asistente virtual de EcoMarket.")
    print("Puedes consultar productos, sucursales, empleados o el menú.\n")

    while True:
        user_input = input("Tú: ")
        salida = procesar_entrada(user_input)

        if salida == "SALIR":
            print("👋 Gracias por usar el asistente de EcoMarket. ¡Hasta pronto!")
            break
        else:
            print(salida)


In [7]:
main()


👋 ¡Bienvenido! Soy el asistente virtual de EcoMarket.
Puedes consultar productos, sucursales, empleados o el menú.


📦 Productos disponibles:

Sucursal: EcoMarket Central (Santiago Centro)
 - Manzanas (SKU: A001, Cantidad: 50)
 - Peras (SKU: A002, Cantidad: 35)
 - Plátanos (SKU: A003, Cantidad: 60)
 - Naranjas (SKU: A004, Cantidad: 40)

Sucursal: EcoMarket Norte (Antofagasta)
 - Pan Integral (SKU: B001, Cantidad: 45)
 - Leche Descremada (SKU: B002, Cantidad: 30)
 - Yogurt Natural (SKU: B003, Cantidad: 25)
 - Queso Fresco (SKU: B004, Cantidad: 20)

Sucursal: EcoMarket Sur (Concepción)
 - Arroz (SKU: C001, Cantidad: 100)
 - Fideos (SKU: C002, Cantidad: 80)
 - Aceite de Girasol (SKU: C003, Cantidad: 40)
 - Harina Integral (SKU: C004, Cantidad: 55)

Sucursal: EcoMarket Oriente (Las Condes)
 - Tomates (SKU: D001, Cantidad: 70)
 - Zanahorias (SKU: D002, Cantidad: 65)
 - Lechuga (SKU: D003, Cantidad: 50)
 - Paltas (SKU: D004, Cantidad: 30)

Sucursal: EcoMarket Poniente (Maipú)
 - Pollo Entero